In [1]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from scipy.signal import convolve2d

In [4]:
class ImageBlender:
    """A class for seamlessly blending two images using Laplacian pyramid blending technique.
    
    This implementation follows the method introduced by Burt and Adelson (1983). The approach:
    1. Decomposes input images into Laplacian pyramids
    2. Creates a Gaussian pyramid for the mask
    3. Blends corresponding levels of the pyramids
    4. Reconstructs the final image from the blended pyramid
    
    Parameters
    ----------
    img1 : PIL.Image
        First input image to blend
    img2 : PIL.Image
        Second input image to blend
    levels : int
        Number of pyramid levels for decomposition
    sigma : float
        Standard deviation for Gaussian filtering
    """
    
    def __init__(self, img1, img2, levels, sigma):
        # Resize images to standard size for consistency
        img1 = img1.resize((512, 512))
        img2 = img2.resize((512, 512))

        self._img1 = np.array(img1)
        self._img2 = np.array(img2)
        self._num_levels = levels
        self._mask = None
        self._blended_image = None
        self._sigma = sigma

        # Initialize pyramid storage
        self._gaussian_pyramids_img1 = []
        self._gaussian_pyramids_img2 = []
        self._laplacian_pyramids_img1 = []
        self._laplacian_pyramids_img2 = []
        self._mask_pyramid = []
        self._blended_pyramid = []
    
    def blend_images(self):
        """Execute the complete blending pipeline."""
        self.build_pyramids()
        self.blend_pyramids()
        self.reconstruct()

    def build_pyramids(self):
        """Construct Gaussian and Laplacian pyramids for both images and the mask."""
        self._gaussian_pyramid_img1 = self.build_gaussian_pyramid(self._img1, self._num_levels, self._sigma)
        self._gaussian_pyramid_img2 = self.build_gaussian_pyramid(self._img2, self._num_levels, self._sigma)
        
        self._laplacian_pyramid_img1 = self.build_laplacian_pyramid(self._gaussian_pyramid_img1)
        self._laplacian_pyramid_img2 = self.build_laplacian_pyramid(self._gaussian_pyramid_img2)
        
        self._mask_pyramid = self.build_mask_pyramid(self._mask, self._num_levels)

    def blend_pyramids(self):
        """Blend corresponding levels of the Laplacian pyramids using the mask."""
        self._blended_pyramid = []
        for l1, l2, mask in zip(self._laplacian_pyramid_img1, self._laplacian_pyramid_img2, self._mask_pyramid):
            combined_mask = l1 * mask + l2 * (1 - mask)
            self._blended_pyramid.append(combined_mask)
        
    def reconstruct(self):
        """Reconstruct the final image from the blended pyramid."""
        image = self._blended_pyramid[-1]
        for i in range(len(self._blended_pyramid) - 2, -1, -1):
            image = self.upsample_image(image)
            image = image + self._blended_pyramid[i]
        self._blended_image = image

    def gaussian_kernel(self, sigma):
        """Create a 2D Gaussian kernel.
        
        Parameters
        ----------
        sigma : float
            Standard deviation of the Gaussian

        Returns
        -------
        ndarray
            2D normalized Gaussian kernel
        """
        x = y = np.arange(-sigma//2 + 1, sigma//2 + 1)
        x, y = np.meshgrid(x, y)
        kernel = np.exp(-(x**2 + y**2) / (2 * sigma**2))
        return kernel / kernel.sum()

    def gaussian_filter(self, image, sigma):
        """Apply Gaussian filtering to an image.
        
        Parameters
        ----------
        image : ndarray
            Input image to filter
        sigma : float
            Standard deviation for Gaussian kernel

        Returns
        -------
        ndarray
            Filtered image
        """
        kernel = self.gaussian_kernel(sigma)

        if image.ndim == 2:  # for the mask which is black and white
            filtered_image = convolve2d(image, kernel, mode='same')
        else:  # for the actual images which are RGB
            filtered_channels = []
            for c in range(image.shape[2]):
                filtered_channel = convolve2d(image[:, :, c], kernel, mode='same')
                filtered_channels.append(filtered_channel)
            filtered_image = np.stack(filtered_channels, axis=2)
        return filtered_image
    
    def build_gaussian_pyramid(self, image, levels, sigma):
        """Build a Gaussian pyramid by successive smoothing and downsampling.
        
        Parameters
        ----------
        image : ndarray
            Input image
        levels : int
            Number of pyramid levels
        sigma : float
            Gaussian smoothing parameter

        Returns
        -------
        list of ndarray
            Gaussian pyramid levels from largest to smallest
        """
        gaussian_pyramid = [image]
        for _ in range(levels):
            image = self.gaussian_filter(image, sigma)
            image = image[::2, ::2]  # downsamples, takes every other pixel
            gaussian_pyramid.append(image)
        return gaussian_pyramid
    
    def upsample_image(self, image, sigma=1):
        """Double the size of an image using Gaussian interpolation.
        
        Parameters
        ----------
        image : ndarray
            Input image to upsample
        sigma : float, optional
            Smoothing for interpolation, by default 1

        Returns
        -------
        ndarray
            Upsampled image
        """
        rows, cols, channels = image.shape
        upsampled_image = np.zeros((rows * 2, cols * 2, channels))
        upsampled_image[::2, ::2] = image
        upsampled_image = self.gaussian_filter(upsampled_image, sigma)
        return upsampled_image
    
    def build_laplacian_pyramid(self, gaussian_pyramid):
        """Build a Laplacian pyramid from a Gaussian pyramid.
        
        Parameters
        ----------
        gaussian_pyramid : list of ndarray
            Input Gaussian pyramid

        Returns
        -------
        list of ndarray
            Laplacian pyramid levels
        """
        laplacian_pyramid = []
        for i in range(len(gaussian_pyramid) - 1):
            upsampled_image = self.upsample_image(gaussian_pyramid[i + 1])
            laplacian = gaussian_pyramid[i] - upsampled_image
            laplacian_pyramid.append(laplacian)
        laplacian_pyramid.append(gaussian_pyramid[-1])  # last level
        return laplacian_pyramid

    def build_mask_pyramid(self, mask, levels):
        """Build a Gaussian pyramid for the blending mask.
        
        Parameters
        ----------
        mask : ndarray
            Input binary mask image
        levels : int
            Number of pyramid levels

        Returns
        -------
        list of ndarray
            Gaussian pyramid of the mask
        """
        mask_pyramid = [mask]
        for _ in range(levels):
            mask = self.gaussian_filter(mask, sigma=1)
            mask = mask[::2, ::2]  # downsample
            mask_pyramid.append(mask)
        return mask_pyramid

    def load_mask_image(self, mask_image_path, sigma=20):
        """Load and prepare a mask image for blending.
        
        Parameters
        ----------
        mask_image_path : str
            Path to the mask image file
        sigma : float, optional
            Smoothing parameter for mask edges, by default 20

        Returns
        -------
        ndarray
            Processed blending mask
        """
        mask_img = Image.open(mask_image_path).convert('L')
        mask_img = mask_img.resize((self._img1.shape[1], self._img1.shape[0]))
        mask = np.array(mask_img) / 255.0
        mask = self.gaussian_filter(mask, sigma)
        self._mask = mask[:, :, np.newaxis]
        return self._mask

In [ ]:
# Example use, must upload own images
img1 = Image.open('img1.jpg').convert('RGB')
img2 = Image.open('img2.jpg').convert('RGB')

blender = ImageBlender(img1, img2, levels=4, sigma=10)

mask = blender.load_mask_image('mask.jpg')  # mask should be black in white, constructed using GIMP or a similar photoshop software

plt.imshow(mask)
plt.axis('off')
plt.show()

blender.blend_images()  # calls build_pyramids() -> blend_pyramids() -> reconstruct()

plt.imshow(blender.blended_image.astype(np.uint8))  # floats need to be converted to ints in order for colors to properly display
plt.axis('off')
plt.show()